In [8]:
import pandas as pd

In [9]:
pwd

u'/Users/aurelperianu/Documents/Medicare_payments_classifier'

In [16]:
#data_all=pd.read_csv('Medicare_Data/2012MedicareProviderPayment.txt', delimiter='\t')
provider_all=pd.read_csv('Medicare_Data/Physician_Compare_2014/National_Downloadable_File.csv')

/Users/aurelperianu/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (15,25,26,28,29,30,31,32,33,34,36,41) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [29]:
#fct rename columns lower letters; remove spaces
def lower_columns(pdx):
    cols = pdx.columns.tolist()
    cols = [col.lower() for col in cols]
    pdx.columns = cols
    pdx.rename(columns=lambda x: x.strip().replace(" ", "_"), inplace=True)
    return pdx
provider_all=lower_columns(provider_all)

In [30]:
#add zip code
provider_all['zip_code']=provider_all['zip_code'].astype(str)
provider_all['zip_base']=provider_all['zip_code'].str.slice(0,5)

#data_all['nppes_provider_zip_base'].unique()

In [63]:
provider_all['professional_accepts_medicare_assignment']=provider_all['professional_accepts_medicare_assignment'].astype(str)

In [64]:
#select WA state providers
provider_all['state']=provider_all['state'].astype(str)
#provider_wa['professional_accepts_medicare_assignment'=='M']
provider_wa=provider_all[provider_all['state']=='WA']
provider_wa.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58889 entries, 38 to 2247052
Data columns (total 43 columns):
npi                                                                   58889 non-null int64
pac_id                                                                58889 non-null int64
professional_enrollment_id                                            58889 non-null object
last_name                                                             58889 non-null object
first_name                                                            58885 non-null object
middle_name                                                           48962 non-null object
suffix                                                                729 non-null object
gender                                                                58889 non-null object
credential                                                            20302 non-null object
medical_school_name                                             

In [32]:
provider_wa.head().T

,38,99,142,168,174
npi,1023030962,1720088685,1295989689,1922016203,1841608809
pac_id,345315404,8921040635,2466513460,648178822,244540201
professional_enrollment_id,I20080825000311,I20050524000096,I20081209000764,I20090401000423,I20151104001076
last_name,BASTRON,LITTMAN,JOHNSON,ULLMAN,MAHER
first_name,MICHAEL,THOMAS,ERIKA,RONALD,ANITA
middle_name,K,A,L,H,D
suffix,NaN,NaN,NaN,NaN,NaN
gender,M,M,F,M,F
credential,NaN,AU,NaN,MD,NaN
medical_school_name,PALMER COLLEGE CHIROPRACTIC - WEST SUNNYVALE,OTHER,OTHER,WASHINGTON UNIVERSITY SCHOOL OF MEDICINE,OTHER


In [73]:
drop_col=['professional_accepts_medicare_assignment','line_2_street_address','marker_of_address_line_2_suppression']

In [72]:
def drop_columns(col_remove,pdx):
    columns=list(pdx.columns.values)
    for col in col_remove:
        if col in columns:
            pdx.drop(col, axis=1, inplace=True)
    return pdx

In [74]:
provider_all=drop_columns(drop_col, provider_all)

In [ ]:
#transform secondary speciality in counts
#transform hospital_affiliation in counts
#change col names

In [75]:
provider_wa=provider_all[provider_all['state']=='WA']
provider_wa.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58889 entries, 38 to 2247052
Data columns (total 40 columns):
npi                                                                   58889 non-null int64
pac_id                                                                58889 non-null int64
professional_enrollment_id                                            58889 non-null object
last_name                                                             58889 non-null object
first_name                                                            58885 non-null object
middle_name                                                           48962 non-null object
suffix                                                                729 non-null object
gender                                                                58889 non-null object
credential                                                            20302 non-null object
medical_school_name                                             

In [68]:
#provider_wa['professional_accepts_medicare_assignment'=='M']
#x=provider_wa[provider_wa['professional_accepts_medicare_assignment']=='M']

In [79]:
provider_wa['hospital_affiliation_lbn_5'].value_counts()

PROVIDENCE REGIONAL MEDICAL CENTER EVERETT            220
SWEDISH ISSAQUAH                                      188
JEFFERSON HEALTHCARE                                  167
SWEDISH EDMONDS HOSPITAL                              163
ST CLARE HOSPITAL                                     160
NEWPORT COMMUNITY HOSPITAL                            141
ST JOSEPH MEDICAL CENTER                              141
ST ANTHONY HOSPITAL                                   139
ST FRANCIS COMMUNITY HOSPITAL                         132
PROVIDENCE ST JOSEPH HOSPITAL                         122
SWEDISH MEDICAL CENTER                                121
SWEDISH MEDICAL CENTER / CHERRY HILL                  121
TACOMA GENERAL ALLENMORE HOSPITAL                     108
EVERGREENHEALTH MEDICAL CENTER                        105
LINCOLN HOSPITAL                                      103
GRAYS HARBOR COMMUNITY HOSPITAL                       100
UNIVERSITY OF WASHINGTON MEDICAL CTR                   84
DEACONESS HOSP